In [1]:
from polished import pmain
import pandas            as pd
import nb_utils          as nbu
#import matplotlib.pyplot as plt
#%matplotlib inline

p = pmain.create_polished( 'postgresql' )
p.initiate_db()
mst = p.masters
pc = p.prrf_current
po = p.prrf_old
pa = p.prrf_project_aue
pg = p.prrf_project_gh

In [2]:
# 1. получение базовых DataFrame-ов
df_data, df_index, df_cols = pc.prrf_current_df
df_current = pd.DataFrame( df_data, index=df_index, columns=df_cols )

df_data, df_index, df_cols = po.prrf_old_df
df_old = pd.DataFrame( df_data, index=df_index, columns=df_cols )

df_data, df_index, df_cols = pa.prrf_project_aue_df
df_project_aue = pd.DataFrame( df_data, index=df_index, columns=df_cols )

df_data, df_index, df_cols = pg.prrf_project_gh_df
df_project_gh = pd.DataFrame( df_data, index=df_index, columns=df_cols )

In [3]:
# 2. prrf_current добавление новых колонок + разыменование справочников + переименование колонок +
#    перестановка колонок местами
df_current['ruscutname']     = df_current.apply( lambda row: mst.get_ruscut_name( row['ruscut_id'] ),         axis=1 )
df_current['russzgrpname']   = df_current.apply( lambda row: mst.get_russzgrp_name( row['russzgrp_id'] ),     axis=1 )
df_current['ruscolorname']   = df_current.apply( lambda row: mst.get_ruscolor_name( row['ruscolor_id'] ),     axis=1 )
df_current['rusclarityname'] = df_current.apply( lambda row: mst.get_rusclarity_name( row['rusclarity_id'] ), axis=1 )
df_current.rename( columns={'rusprice':'curr_rusprice'}, inplace=True )
df_current = df_current[ [ 'ruscutname', 'ruscut_id', 'russzgrpname', 'russzgrp_id',
                           'ruscolorname', 'ruscolor_id', 'rusclarityname', 'rusclarity_id',
                           'curr_rusprice' ]]

In [4]:
#3. prrf_old, prrf_project_aue, prrf_project_gh переименование колонок, сложение с prrf_current - "цена к цене", экспорт в xlsx
df_old.rename( columns={ 'rusprice': 'old_rusprice' }, inplace=True )
df_project_aue.rename( columns={ 'rusprice': 'aue_rusprice' }, inplace=True )
df_project_gh.rename( columns={ 'rusprice': 'gh_rusprice' }, inplace=True )
df_price_analisys = pd.concat( [ df_current, df_project_aue[ 'aue_rusprice' ], df_project_gh[ 'gh_rusprice' ]], axis=1 )
nbu.pandas_to_xlsx( df_price_analisys, 'd:\py_dev\work\out_files\prrf_compare.xlsx', 'prrf_prices')

In [5]:
#4. prrf_aue - вывод одного квадрата прейскуранта с заменой меток по вертикали и по горизонтали
#TO-DO: отладить
#0.30-0.39
df_pivot = df_project_aue[ ( df_project_aue.ruscut_id==3 ) & ( df_project_aue.russzgrp_id==16 ) ]   \
                         [['ruscolor_id', 'rusclarity_id', 'aue_rusprice']]. \
                         pivot_table( index=[ 'ruscolor_id' ], 
                                      columns=[ 'rusclarity_id' ],
                                      values='aue_rusprice',
                                      aggfunc='sum'
                                    )
df_pivot_r = pd.DataFrame( df_pivot.values,
                           index=[ p.masters.get_ruscolor_name( ind ) for ind in df_pivot.index ],
                           columns=[ p.masters.get_rusclarity_name( col ) for col in df_pivot.columns ]
                         )
df_pivot_r

,1,2,3,4,5,6,7,7А,8,9,10,11,12
1,2400.00,2100.00,1880.00,1700.00,1540.00,1345.00,1170.00,1010.00,880.00,700.00,550.00,370.00,100.00
2,2200.00,1920.00,1725.00,1570.00,1440.00,1265.00,1110.00,960.00,840.00,670.00,530.00,360.00,100.00
3,2050.00,1810.00,1640.00,1500.00,1380.00,1210.00,1060.00,920.00,810.00,645.00,515.00,350.00,95.00
4,1920.00,1710.00,1550.00,1420.00,1310.00,1150.00,1010.00,885.00,780.00,625.00,500.00,340.00,90.00
5,1780.00,1590.00,1440.00,1330.00,1230.00,1085.00,960.00,845.00,745.00,600.00,485.00,330.00,85.00
6,1570.00,1410.00,1300.00,1210.00,1125.00,1000.00,885.00,780.00,695.00,570.00,465.00,320.00,80.00
6.1,1450.00,1310.00,1220.00,1140.00,1065.00,945.00,840.00,740.00,660.00,540.00,445.00,310.00,80.00
7,1330.00,1205.00,1150.00,1075.00,1005.00,890.00,790.00,695.00,625.00,515.00,430.00,300.00,75.00
8.1,1160.00,1055.00,1010.00,945.00,880.00,780.00,690.00,615.00,555.00,470.00,405.00,285.00,70.00
8.2,985.00,900.00,860.00,805.00,750.00,670.00,600.00,540.00,490.00,420.00,370.00,265.00,65.00


In [6]:
#ds1 = df_pivot_r.loc['1']